In [33]:
#import the necessary libraries
import numpy as np
import pandas as pd
import plotly.express as px


In [34]:
#load the combined data
df = pd.read_csv('combined_data.csv')
df.head()

,OrderNumber,QuantityOrdered,PriceEach,OrderLineNumber,Sales,Exact Sales,Status,QTR_ID,DAY_ID,MONTH_ID,...,Country,Territory,Branch,CostPrice,ProductLine,MSRP,ContactName,Total Profit,Discount,Discount Rate
0,10100,49,34.47,1,1689.03,1689.03,Shipped,1,6,1,...,USA,NorthA,Skates Road,22.4055,Vintage Cars,41,Valarie Young,591.1605,6.53,15.926829
1,10100,50,67.80,2,3390.00,3390.00,Shipped,1,6,1,...,USA,NorthA,Bergenville,44.0700,Vintage Cars,60,Valarie Young,1186.5000,-7.80,-13.000000
2,10100,30,100.00,3,5151.00,3000.00,Shipped,1,6,1,...,USA,NorthA,Bergenville,65.0000,Vintage Cars,170,Valarie Young,1050.0000,70.00,41.176471
3,10100,22,86.51,4,1903.22,1903.22,Shipped,1,6,1,...,USA,NorthA,Skates Road,56.2315,Vintage Cars,92,Valarie Young,666.1270,5.49,5.967391
4,10101,26,100.00,1,3773.38,2600.00,Shipped,1,9,1,...,Germany,EMEA,Bergenville,65.0000,Vintage Cars,168,Roland Keitel,910.0000,68.00,40.476190


- Perform customer segmentation(RFM SEGMENTATION)
1. Perform RFM- Recency, Frequency and Monetary 
- Recency: How recently did a customer make a purchase? Helps identify active vs. dormant customers.
- Frequency: How often does a customer buy products? Identifies loyal customers with frequent purchases.
- Monetary: How much does a customer spend? Shows high-value customers who contribute more revenue.

In [35]:
#perform customer segmentation
#get the recency, Frequency and Monetary value of each customer
#first, combine the MONTH_ID, YEAR_ID, DAY_ID to form a single date column
df['Date'] = df[['YEAR_ID','MONTH_ID','DAY_ID']].apply(lambda x: '-'.join(x.values.astype(str)), axis=1)
df.head()

,OrderNumber,QuantityOrdered,PriceEach,OrderLineNumber,Sales,Exact Sales,Status,QTR_ID,DAY_ID,MONTH_ID,...,Territory,Branch,CostPrice,ProductLine,MSRP,ContactName,Total Profit,Discount,Discount Rate,Date
0,10100,49,34.47,1,1689.03,1689.03,Shipped,1,6,1,...,NorthA,Skates Road,22.4055,Vintage Cars,41,Valarie Young,591.1605,6.53,15.926829,2022-1-6
1,10100,50,67.80,2,3390.00,3390.00,Shipped,1,6,1,...,NorthA,Bergenville,44.0700,Vintage Cars,60,Valarie Young,1186.5000,-7.80,-13.000000,2022-1-6
2,10100,30,100.00,3,5151.00,3000.00,Shipped,1,6,1,...,NorthA,Bergenville,65.0000,Vintage Cars,170,Valarie Young,1050.0000,70.00,41.176471,2022-1-6
3,10100,22,86.51,4,1903.22,1903.22,Shipped,1,6,1,...,NorthA,Skates Road,56.2315,Vintage Cars,92,Valarie Young,666.1270,5.49,5.967391,2022-1-6
4,10101,26,100.00,1,3773.38,2600.00,Shipped,1,9,1,...,EMEA,Bergenville,65.0000,Vintage Cars,168,Roland Keitel,910.0000,68.00,40.476190,2022-1-9


In [36]:
#change the date column to datetime
df['Date'] = pd.to_datetime(df['Date'])


In [37]:
#calculate the recency, frequency and monetary value of each customer
#get the most recent date in the dataset
most_recent_date = df['Date'].max()

# Calculate Recency as the difference in days between the most recent date and each customer's last purchase date
df['Recency'] = df.groupby('CustomerName')['Date'].transform(lambda x: (most_recent_date - x.max()).days)   

# Calculate Frequency as the number of unique purchase dates per customer. We use transform to broadcast the result back to the original dataframe
df['Frequency'] = df.groupby('CustomerName')['Date'].transform('nunique')

# Calculate Monetary as the total Exact Sales per customer.
df['Monetary'] = df.groupby('CustomerName')['Exact Sales'].transform('sum')

#drop the duplicate rows
rfm = df[['CustomerName', 'Recency','Frequency','Monetary']].drop_duplicates()




In [38]:
rfm.head(15)

,CustomerName,Recency,Frequency,Monetary
0,Online Diecast Creations Co.,209,3,101428.50
4,Blauer See Auto Co.,208,4,66945.37
8,Vitachrome Inc.,208,3,67511.37
10,Baane Mini Imports,208,4,95223.90
26,Euro Shopping Channel,0,25,766195.05
39,Danish Wholesale Imports,46,5,115268.84
54,Rovelli Gifts,201,3,127143.64
72,Land of Toys Inc.,198,4,138798.17
80,Cruz & Sons Co.,197,3,80291.17
96,Motor Mint Distributors Inc.,196,3,63091.46


In [39]:
#sort the data by recency, frequency and monetary value
rfm = rfm.sort_values(by=['Recency','Frequency','Monetary'], ascending=[True, False, False])
rfm.head(15)

,CustomerName,Recency,Frequency,Monetary
26,Euro Shopping Channel,0,25,766195.05
1529,La Rochelle Gifts,0,4,153909.82
1503,Diecast Classics Inc.,1,4,96378.27
1112,Petit Auto,1,3,68568.18
126,Mini Gifts Distributors Ltd.,2,17,530587.19
323,Souveniers And Things Co.,2,4,133223.38
159,Salzburg Collectables,14,4,126287.85
696,L ordine Souveniers,21,3,109614.22
865,Australian Collectables Ltd,22,3,56139.62
1449,Gifts4AllAges.com,25,3,77547.62


In [40]:
#apply the RFM scoring as before
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1]).astype(int)
# Use pd.cut() with custom bin ranges
rfm['F_Score'] = pd.cut(rfm['Frequency'], bins=4, labels=[1, 2, 3, 4]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4]).astype(int)
rfm['RFM_Score'] = rfm['R_Score'] * 100 + rfm['F_Score'] * 10 + rfm['M_Score']

print(rfm[['CustomerName', 'R_Score', 'F_Score', 'M_Score', 'RFM_Score']])

                        CustomerName  R_Score  F_Score  M_Score  RFM_Score
26             Euro Shopping Channel        4        4        4        444
1529               La Rochelle Gifts        4        1        4        414
1503           Diecast Classics Inc.        4        1        3        413
1112                      Petit Auto        4        1        2        412
126     Mini Gifts Distributors Ltd.        4        3        4        434
...                              ...      ...      ...      ...        ...
729         Daedalus Designs Imports        1        1        2        112
427         Signal Collectibles Ltd.        1        1        1        111
930      West Coast Collectables Co.        1        1        1        111
818   Double Decker Gift Stores  Ltd        1        1        1        111
537       Men  R  US Retailers  Ltd.        1        1        1        111

[92 rows x 5 columns]


In [41]:
df['CustomerName'].nunique()

92

In [42]:
#group the customers into segments
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 300:
        return 'Best Customers'
    elif ((df['RFM_Score'] >= 200) and (df['RFM_Score'] < 300)):
        return 'Average Customers'
    elif ((df['RFM_Score'] >= 100) and (df['RFM_Score'] < 200)):
        return 'Potential Customers'
    else:
        return 'Churned Customers'
    

# Create a new variable RFM_Level
rfm['RFM_Level'] = rfm.apply(rfm_level, axis=1)

rfm.head()  


,CustomerName,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,RFM_Level
26,Euro Shopping Channel,0,25,766195.05,4,4,4,444,Best Customers
1529,La Rochelle Gifts,0,4,153909.82,4,1,4,414,Best Customers
1503,Diecast Classics Inc.,1,4,96378.27,4,1,3,413,Best Customers
1112,Petit Auto,1,3,68568.18,4,1,2,412,Best Customers
126,Mini Gifts Distributors Ltd.,2,17,530587.19,4,3,4,434,Best Customers


In [43]:
#plot the customer segments
#fig = px.scatter(rfm, x='Recency', y='Frequency', color='RFM_Level', title='RFM Segments')
#fig.show()


In [44]:
# plot the customer segments
#fig = px.scatter(rfm, x='Recency', y='Monetary', color='RFM_Level', title='RFM Segments')
#fig.show()

In [45]:
#plot bar chart of th breakdown of customers in the best customers segment
#fig = px.bar(rfm['RFM_Level'].value_counts().sort_values(ascending=True).reset_index(), x='count', y='RFM_Level', title='Breakdown of Customers in Each Segment')
#fig.show()

## Time Analysis for the Customers

- Extract the names of days from the Date column.

In [46]:
#extract the day of the week from the date column
df['Day'] = df['Date'].dt.day_name()
df.head()

,OrderNumber,QuantityOrdered,PriceEach,OrderLineNumber,Sales,Exact Sales,Status,QTR_ID,DAY_ID,MONTH_ID,...,MSRP,ContactName,Total Profit,Discount,Discount Rate,Date,Recency,Frequency,Monetary,Day
0,10100,49,34.47,1,1689.03,1689.03,Shipped,1,6,1,...,41,Valarie Young,591.1605,6.53,15.926829,2022-01-06,209,3,101428.50,Thursday
1,10100,50,67.80,2,3390.00,3390.00,Shipped,1,6,1,...,60,Valarie Young,1186.5000,-7.80,-13.000000,2022-01-06,209,3,101428.50,Thursday
2,10100,30,100.00,3,5151.00,3000.00,Shipped,1,6,1,...,170,Valarie Young,1050.0000,70.00,41.176471,2022-01-06,209,3,101428.50,Thursday
3,10100,22,86.51,4,1903.22,1903.22,Shipped,1,6,1,...,92,Valarie Young,666.1270,5.49,5.967391,2022-01-06,209,3,101428.50,Thursday
4,10101,26,100.00,1,3773.38,2600.00,Shipped,1,9,1,...,168,Roland Keitel,910.0000,68.00,40.476190,2022-01-09,208,4,66945.37,Sunday


In [47]:
#get the total sales per day of the week
#total_sales_per_day = df.groupby('Day')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
#plot the total sales per day of the week
#fig = px.bar(total_sales_per_day, x='Day', y='Exact Sales', title='Total Sales per Day of the Week')
#fig.show()


- Analyze the days with highest sales for the best_customers segments

- Best Customers

In [48]:
#analyze the time series of these customers
#first, get the data for the best customers
#best_customers = best_customers['CustomerName'].values
#get the complete data of customers in the best customers segment
#best_customers_data = df[df['CustomerName'].isin(best_customers)]
#best_customers_data.head()

## Geography Segmentation 
- Analyze customer location data to understand which regions generate the most revenue, have higher order frequencies, or larger average order values.

In [49]:
# Grouping data by city, country, and territory to get aggregates
region_data = df.groupby(['CustomerName','City', 'Country', 'Territory']).agg({
    'Exact Sales': 'sum',
    'Total Profit': 'sum',
    'QuantityOrdered': 'count'
}).reset_index()
# Calculate the average order value- the average amount spent per order
region_data['Average_Order_Value'] = region_data['Exact Sales'] / region_data['QuantityOrdered']



In [50]:
region_data.head()

,CustomerName,City,Country,Territory,Exact Sales,Total Profit,QuantityOrdered,Average_Order_Value
0,AV Stores Co.,Manchester,UK,EMEA,136873.25,47905.6375,51,2683.789216
1,Alpha Cognac,Toulouse,France,EMEA,58885.28,20609.8480,20,2944.264000
2,Amica Models & Co.,Torino,Italy,EMEA,72645.01,25425.7535,26,2794.038846
3,Anna s Decorations Ltd,North Sydney,Australia,APAC,121260.41,42441.1435,46,2636.095870
4,Atelier graphique,Nantes,France,EMEA,21213.16,7424.6060,7,3030.451429


In [51]:
#segment the regions in quartiles
# Defining segmentation conditions
conditions = [
    (region_data['Exact Sales'] > region_data['Exact Sales'].quantile(0.75)) & 
    (region_data['Total Profit'] > region_data['Total Profit'].quantile(0.75)),
    (region_data['Exact Sales'] < region_data['Exact Sales'].quantile(0.25)) & 
    (region_data['Total Profit'] < region_data['Total Profit'].quantile(0.25))
]
# Defining the labels for each segment
labels = ['Top Region', 'Underperforming Region']
# Creating the Segment column. np.select- assigns the label based on the conditions
region_data['Segment'] = np.select(conditions, labels, default='Average Region')


In [52]:
region_data.head()

,CustomerName,City,Country,Territory,Exact Sales,Total Profit,QuantityOrdered,Average_Order_Value,Segment
0,AV Stores Co.,Manchester,UK,EMEA,136873.25,47905.6375,51,2683.789216,Top Region
1,Alpha Cognac,Toulouse,France,EMEA,58885.28,20609.8480,20,2944.264000,Average Region
2,Amica Models & Co.,Torino,Italy,EMEA,72645.01,25425.7535,26,2794.038846,Average Region
3,Anna s Decorations Ltd,North Sydney,Australia,APAC,121260.41,42441.1435,46,2636.095870,Top Region
4,Atelier graphique,Nantes,France,EMEA,21213.16,7424.6060,7,3030.451429,Underperforming Region


In [53]:
# Creating a map visualization
fig = px.scatter_geo(region_data, 
                     locations="Country",
                     locationmode="country names",
                     color="Segment",
                     hover_name="City",
                     size="Exact Sales",
                     projection="natural earth",
                     title="Geographic Segmentation by Sales and Profit")
fig.show()

- Perform Kmeans clustering

In [54]:
#perform clustering - to group customers based on their RFM scores
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#select the RFM scores
rfm_scores = rfm[['Recency', 'Frequency', 'Monetary']]
#standardize the data
scaler = StandardScaler()
#apply the scaler
rfm_scores_scaled = scaler.fit_transform(rfm_scores)

#instantiate the KMeans model
kmeans = KMeans(n_clusters=3, random_state=42)
#fit the model
kmeans.fit(rfm_scores_scaled)
#predict the clusters
rfm['Cluster'] = kmeans.predict(rfm_scores_scaled)  
rfm['Cluster'].value_counts()

Cluster
0    78
1    12
2     2
Name: count, dtype: int64

In [55]:
#combine the customer segments with the clusters
rfm['Cluster'] = rfm['Cluster'].map({0:'Cluster 1', 1:'Cluster 2', 2:'Cluster 3'})
rfm.head()


,CustomerName,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,RFM_Level,Cluster
26,Euro Shopping Channel,0,25,766195.05,4,4,4,444,Best Customers,Cluster 3
1529,La Rochelle Gifts,0,4,153909.82,4,1,4,414,Best Customers,Cluster 1
1503,Diecast Classics Inc.,1,4,96378.27,4,1,3,413,Best Customers,Cluster 1
1112,Petit Auto,1,3,68568.18,4,1,2,412,Best Customers,Cluster 1
126,Mini Gifts Distributors Ltd.,2,17,530587.19,4,3,4,434,Best Customers,Cluster 3


In [56]:
#display customers in each cluster
fig = px.scatter(rfm, x='Recency', y='Frequency', color='Cluster', title='Clusters of Customers', labels={'Recency':'Recency (Days)', 'Frequency':'Frequency (No. of Orders)'})
fig.show()

In [57]:
# for each cluster, plot the customerName and monetary value
fig = px.bar(rfm.groupby('Cluster')['Monetary'].sum().sort_values(ascending=False).reset_index(), x='Cluster', y='Monetary', title='Total Sales by Cluster')
fig.update_layout(
    bargap=0.5,
    xaxis_title='Cluster',
    yaxis_title='Monetary Input ($)',
    

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)

)
fig.show()
    

In [58]:
#display all customers in each cluster
#cluster 1
cluster1 = rfm[rfm['Cluster']=='Cluster 1'].sort_values(by='Monetary', ascending=False)
fig = px.bar(cluster1.head(10), x='Monetary', y='CustomerName', title='Cluster 1(Top 10, Low Recency and Low Frequency)', category_orders={'CustomerName': cluster1['CustomerName'].to_list()})
fig.update_layout(
    bargap=0.1,
    xaxis_title='Monetary Input ($)',
    yaxis_title='Customer Name',
    

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()


In [59]:
#bottom 10 customers in cluster 1 make it a vertical bar chart
cluster1_bottom = rfm[rfm['Cluster']=='Cluster 1'].sort_values(by='Monetary', ascending=True)
fig = px.bar(cluster1_bottom.head(10), x='Monetary', y='CustomerName', title='Cluster 1( Bottom 10, Low Recency and Low Frequency)')
fig.update_layout(
    bargap=0.1,
    xaxis_title='Monetary Input ($)',
    yaxis_title='Customer Name',
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)
)
fig.show()

In [60]:
#cluster 1 time series
cluster1_data = df[df['CustomerName'].isin(cluster1['CustomerName'].values)]
cluster1_seasonality = cluster1_data.groupby(['YEAR_ID','MONTH_ID'])['Exact Sales'].sum().reset_index()
fig = px.line(cluster1_seasonality, x='MONTH_ID', y='Exact Sales', color='YEAR_ID', title='Seasonality of Cluster 1')
fig.show()


In [61]:
#day of the week with the highest sales
cluster1_data = cluster1_data.groupby('Day')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster1_data, x='Day', y='Exact Sales', title='Day of the Week with the Highest Sales(cluster 1)')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Day',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [62]:
#show which products are the most popular in each cluster
#combine the cluster data with the main data
cluster1 = cluster1.merge(df, on='CustomerName')
cluster1_products = cluster1.groupby('ProductLine')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster1_products, x='ProductLine', y='Exact Sales', title='Most Popular Products in Cluster 1')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Product Line',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()


In [64]:
#identify the regions with the highest sales in each cluster
#cluster 1
# Grouping data by city, country, and territory to get aggregates
region_data = df.groupby(['CustomerName','City', 'Country', 'Territory']).agg({
    'Exact Sales': 'sum',
    'Total Profit': 'sum',
    'QuantityOrdered': 'count'
}).reset_index()
# Calculate the average order value- the average amount spent per order
region_data['Average_Order_Value'] = region_data['Exact Sales'] / region_data['QuantityOrdered']

cluster1 = cluster1.merge(region_data, on='CustomerName')
cluster1_regions = cluster1.groupby('Country')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster1_regions, x='Country', y='Exact Sales', title='Regions with the Highest Sales in Cluster 1')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Country',
    yaxis_title='Exact Sales',
    
hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [65]:
#cluster 2      
cluster2 = rfm[rfm['Cluster'] == 'Cluster 2'].sort_values(by='Monetary', ascending=False)
fig = px.bar(cluster2.head(10), x='Monetary', y='CustomerName', title='Cluster 2 (Top 10, High Recency and Low Frequency)',category_orders={'CustomerName': cluster2['CustomerName'].tolist()})
fig.update_layout(
    barmode='stack',
    bargap=0.2,
    xaxis_title='Monetary Input ($)',
    yaxis_title='Customer Name',
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)
#order the bars from the highest to the lowest

fig.show()


In [ ]:
#bottom 10 cluster 2
cluster2_bottom = rfm[rfm['Cluster'] == 'Cluster 2'].sort_values(by='Monetary', ascending=True)
fig = px.bar(cluster2_bottom.head(5), x='Monetary', y='CustomerName',title='Cluster 2 (Bottom 5, High Recency and Low Frequency)')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Monetary Input ($)',
    yaxis_title='Customer Name',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [66]:
#cluster 2 time series
cluster2_data = df[df['CustomerName'].isin(cluster2['CustomerName'].values)]
cluster2_seasonality = cluster2_data.groupby(['YEAR_ID','MONTH_ID'])['Exact Sales'].sum().reset_index()
fig = px.line(cluster2_seasonality, x='MONTH_ID', y='Exact Sales', color='YEAR_ID', title='Seasonality of Cluster 2')
fig.show()


In [67]:
#day of the week with the highest sales
cluster2_data = cluster2_data.groupby('Day')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster2_data, x='Day', y='Exact Sales', title='Day of the Week with the Highest Sales(cluster2)')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Day',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [68]:
#cluster 2
cluster2 = cluster2.merge(df, on='CustomerName')
cluster2_products = cluster2.groupby('ProductLine')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster2_products, x='ProductLine', y='Exact Sales', title='Most Popular Products in Cluster 2')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Product Line',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [ ]:
#cluster 2
cluster2 = cluster2.merge(region_data, on='CustomerName')
cluster2_regions = cluster2.groupby('Country')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster2_regions, x='Country', y='Exact Sales', title='Regions with the Highest Sales in Cluster 2')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Country',
    yaxis_title='Exact Sales',
    
hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [ ]:
#cluster 3
cluster3 = rfm[rfm['Cluster']=='Cluster 3'].sort_values(by='Monetary', ascending=False)
fig = px.bar(cluster3, x='CustomerName', y='Monetary',title='Cluster 3(Low Recency and High Frequency)')
fig.update_layout(
    bargap=0.6,
    xaxis_title='Customer Name',
    yaxis_title='Monetary Input ($)',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [ ]:
#cluster 3 time series
cluster3_data = df[df['CustomerName'].isin(cluster3['CustomerName'].values)]
cluster3_seasonality = cluster3_data.groupby(['YEAR_ID','MONTH_ID'])['Exact Sales'].sum().reset_index()
fig = px.line(cluster3_seasonality, x='MONTH_ID', y='Exact Sales', color='YEAR_ID', title='Seasonality of Cluster 3')
fig.show()

In [ ]:
#day of the week with the highest sales
cluster3_data = cluster3_data.groupby('Day')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster3_data, x='Day', y='Exact Sales', title='Day of the Week with the Highest Sales(cluster3)')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Day',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [ ]:
#cluster 3
cluster3 = cluster3.merge(df, on='CustomerName')
cluster3_products = cluster3.groupby('ProductLine')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster3_products, x='ProductLine', y='Exact Sales', title='Most Popular Products in Cluster 3')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Product Line',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [ ]:
#cluster 3
cluster3 = cluster3.merge(region_data, on='CustomerName')
cluster3_regions = cluster3.groupby('Country')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(cluster3_regions, x='Country', y='Exact Sales', title='Regions with the Highest Sales in Cluster 3')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Country',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

- Time Analysis in General 

In [69]:
#TOTAL SALES PER DAY OF THE WEEK
total_sales_per_day = df.groupby('Day')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(total_sales_per_day, x='Day', y='Exact Sales', title='Total Sales per Day of the Week(In General)')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Day',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [71]:
#total product line sales   
total_product_sales = df.groupby('ProductLine')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(total_product_sales, x='ProductLine', y='Exact Sales', title='Total Sales per Product Line(In general)')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Product Line',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)

)
fig.show()

In [72]:
#top 10 countries with highest sales
total_country_sales = df.groupby('Country')['Exact Sales'].sum().sort_values(ascending=False).reset_index()
fig = px.bar(total_country_sales.head(10), x='Country', y='Exact Sales', title='Top 10 Countries with the Highest Sales')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Country',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [73]:
#bottom 10 countries with lowest sales
total_country_sales = df.groupby('Country')['Exact Sales'].sum().sort_values(ascending=True).reset_index()
fig = px.bar(total_country_sales.head(10), x='Country', y='Exact Sales', title='Bottom 10 Countries with the Lowest Sales')
fig.update_layout(
    bargap=0.4,
    xaxis_title='Country',
    yaxis_title='Exact Sales',

hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="Rockwell"
)



)
fig.show()

In [ ]:
#save rfm table as csv
rfm.to_csv('rfm', index=False)